In [56]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

import warnings
warnings.filterwarnings('ignore')

In [11]:
data = pd.read_excel('data/03_결측치처리후_주소지_4천개.xlsx')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head(5)

,ID,위도,경도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,37.483372,127.053437
1,서울특별시 강남구 개포동 개포주공1단지 40,37.483381,127.057068
2,서울특별시 강남구 개포동 개포주공5단지 60,37.487868,127.068988
3,서울특별시 강남구 개포동 개포주공5단지 80,37.487868,127.068988
4,서울특별시 강남구 개포동 개포주공6단지 50,37.488481,127.072502


In [12]:
base = pd.read_excel('data/ID있는버전.xlsx')
base['면적당매매가'] = base['거래금액(만원)'] / base['전용면적(㎡)']
base['저층여부'] = np.where(base['층']<5, '저층', '고층')
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
del base['계약일']
base.drop(base[base['전용면적(㎡)']<=10].index, inplace=True)

base.head(5)

,ID,시군구,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,거래유형,면적당매매가,저층여부,분기
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.000000,저층,2018/1
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,2018-01-01,74000,27,1996,보라매로5길 51,-,493.333333,고층,2018/1
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,2018-01-01,65000,17,2001,여의대방로10길 14,-,812.500000,고층,2018/1
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,2018-01-01,85800,15,2013,상도로 346-2,-,1072.500000,고층,2018/1
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,2018-01-01,104500,12,2013,상도로 346-2,-,870.833333,고층,2018/1


In [13]:
table1 = pd.pivot_table(base, values=['거래금액(만원)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
check_null1 = table1.isna().sum() / 18
check_null1[check_null1 >= 0.5]
remove_cols1 = check_null1[check_null1 >= 0.5].keys()
table1 = table1.drop(remove_cols1, axis=1)
table1.fillna(method='ffill', inplace=True)
table1.fillna(method='bfill', inplace=True)
data1 = table1.transpose()
money = data1.droplevel(axis=0, level=0)
money = money.round(1)
money.head()

분기,2018/1,2018/2,2018/3,2018/4,2019/1,2019/2,2019/3,2019/4,2020/1,2020/2,2020/3,2020/4,2021/1,2021/2,2021/3,2021/4,2022/1,2022/2
ID,,,,,,,,,,,,,,,,,,
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,145000.0,160000.0,160000.0,160000.0,141000.0,154500.0,154500.0,154500.0,157583.3,173900.0,155400.0,155400.0,195000.0,210000.0,215000.0,215000.0,218666.7
서울특별시 강남구 개포동 개포주공1단지 40,145736.8,162587.5,167687.5,150000.0,142500.0,157539.5,187208.3,193705.9,182000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0
서울특별시 강남구 개포동 개포주공5단지 60,136666.7,129500.0,125000.0,125000.0,134500.0,142000.0,159500.0,166000.0,173500.0,167500.0,184800.0,202800.0,202800.0,202800.0,202800.0,202800.0,202800.0,202800.0
서울특별시 강남구 개포동 개포주공5단지 80,177416.7,177416.7,177416.7,141000.0,176000.0,179166.7,194666.7,194666.7,198000.0,186000.0,221714.3,221714.3,221714.3,274333.3,274333.3,282250.0,282250.0,282250.0
서울특별시 강남구 개포동 개포주공6단지 50,125900.0,128000.0,118500.0,118500.0,118000.0,118000.0,148250.0,161500.0,157750.0,148000.0,120250.0,174333.3,189000.0,189000.0,189000.0,189000.0,189000.0,189000.0


In [62]:
dic = {}
number = 0

for i1 in range(18, 22):
    for i2 in range(1, 5):
        dic[f'data{i1}{i2}']=money.iloc[:,number:number+1]
        number += 1

dic['data221'] = money.iloc[:,16:17]
dic['data222'] = money.iloc[:,17:18]

mm = MinMaxScaler()
scaled_dic = {}

scaled_dic['mm_scaled1'] = mm.fit_transform(dic['data181'])
number = 1

for i1 in range(18, 22):
    for i2 in range(1, 5):
        scaled_dic[f'mm_scaled{number}']=mm.fit_transform(dic[f'data{i1}{i2}'])
        number += 1

scaled_dic['mm_scaled17'] = dic['data221']
scaled_dic['mm_scaled18'] = dic['data222']

kmeans = KMeans(n_clusters=5, algorithm='auto', random_state=np.random.RandomState(seed=1))

# kmeans = DBSCAN(eps=0.3, min_samples=10)

number = 1

for i1 in range(18, 22):
    for i2 in range(1, 5):
        clusters = kmeans.fit(scaled_dic[f'mm_scaled{number}'])
        dic[f'data{i1}{i2}'][f'cluster{i1}{i2}'] = clusters.labels_
        number += 1
        
clusters = kmeans.fit(scaled_dic['mm_scaled17'])
dic['data221']['cluster221'] = clusters.labels_
clusters = kmeans.fit(scaled_dic['mm_scaled18'])
dic['data222']['cluster222'] = clusters.labels_

In [63]:
address = pd.merge(dic['data181'], data, left_on='ID', right_on='ID', how='inner')
address

,ID,2018/1,cluster181,위도,경도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,3,37.483372,127.053437
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,3,37.483381,127.057068
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,3,37.487868,127.068988
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,1,37.487868,127.068988
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,3,37.488481,127.072502
...,...,...,...,...,...
4089,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,2,37.60543,127.082936
4090,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,2,37.599447,127.078938
4091,서울특별시 중랑구 중화동 한신아파트 50,31756.2,2,37.595961,127.080314
4092,서울특별시 중랑구 중화동 한신아파트 60,36534.6,2,37.595961,127.080314


In [64]:
address = pd.merge(dic['data182'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data183'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data184'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data191'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data192'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data193'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data194'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data201'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data202'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data203'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data204'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data211'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data212'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data213'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data214'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data221'], address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(dic['data222'], address, left_on='ID', right_on='ID', how='inner')

In [65]:
address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4094 entries, 0 to 4093
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          4094 non-null   object 
 1   2022/2      4094 non-null   float64
 2   cluster222  4094 non-null   int32  
 3   2022/1      4094 non-null   float64
 4   cluster221  4094 non-null   int32  
 5   2021/4      4094 non-null   float64
 6   cluster214  4094 non-null   int32  
 7   2021/3      4094 non-null   float64
 8   cluster213  4094 non-null   int32  
 9   2021/2      4094 non-null   float64
 10  cluster212  4094 non-null   int32  
 11  2021/1      4094 non-null   float64
 12  cluster211  4094 non-null   int32  
 13  2020/4      4094 non-null   float64
 14  cluster204  4094 non-null   int32  
 15  2020/3      4094 non-null   float64
 16  cluster203  4094 non-null   int32  
 17  2020/2      4094 non-null   float64
 18  cluster202  4094 non-null   int32  
 19  2020/1      4094 non-null  

In [ ]:
address.to_excel('data/Big_address.xlsx')

In [61]:
address.to_excel('data/db_Big_address.xlsx')